In [1]:
%pwd

'd:\\Healthbot\\medicobot\\research'

In [2]:
import os
from langchain.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.prompt import PromptTemplate
from loguru import logger

load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

c:\Users\91787\anaconda3\envs\medichatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\91787\AppData\Local\Temp\ipykernel_19888\3401734470.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [4]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medicalchatbot"
index = pc.Index(index_name)

In [5]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [6]:
def search_db(user_query: str) -> list:
    sim_docs =[]
    result = vector_store.similarity_search_with_score(
    user_query, k=3
    )
    for doc in result:
        sim_docs.append(doc[0].page_content)
    
    return sim_docs    

In [7]:
search = search_db(user_query="What is candidiasis?")

In [8]:
search

['GALE ENCYCLOPEDIA OF MEDICINE 2268\nAntifungal drugs, systemic\nGEM - 0001 to 0432 - A  10/22/03 1:42 PM  Page 268',
 'Fungus—A member of a group of simple organ-\nisms that are related to yeast and molds.\nIncontinence—The inability to control the bladder\nor bowel.\nInflammation —Pain, redness, swelling, and heat\nthat usually develop in response to injury or illness.\ning or who plans to become pregnant should check with\nher physician before using a topical antibiotic.\nUnless a physician says to do so, do not use topical\nantibiotics on children under two years of age.\nSide effects',
 'fungi. (Custom Medical Stock Photo. Reproduced by permission.)\nGEM -0433 to 0624 - B  10/22/03 6:08 PM  Page 513']

In [9]:
# ---------------------------- LLM --------------------------------------
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0.5)

In [10]:
def medicalbot_ai(user_query: str, doc_list: list) -> str:
    template = """
    You are a medical consultant AI chatbot. Your role is to provide accurate and reliable answers to user questions based on the provided documents. Use the information from the `doc_list` to address the `user_query` thoroughly and correctly. Ensure that your response is:

    - **Accurate:** Base your answers solely on the information in the provided documents.
    - **Conversational:** Maintain a friendly and approachable tone.
    - **Mature and Consultancy-Oriented:** Present information in a professional and trustworthy manner.

    **Inputs:**
    1. `user_query`: {user_query} The question posed by the user.
    2. `doc_list`: {doc_list} A list of documents containing relevant information related to the user's question.

    **Instructions:**
    - Analyze the `user_query` and identify the key information needed to answer it.
    - Review the `doc_list` to find relevant information that addresses the query.
    - Construct a response that is clear, concise, and directly answers the user's question using the information from the documents.
    - Avoid introducing information not present in the `doc_list`.
    - If the `user_query` have nothing similar to what is in the `doc_list`, return document not found or something in an apologetic way, tell the user to ask for something related to the context.
    - If the  `user_query` is an empty string, respond with "Please provide a valid query.".
    - Maintain a tone that is both professional and empathetic, suitable for a consultancy setting.
    
    Return the answer as the only output. 
    Always make sure that you're returning the answer without any explanation. 
    The output should be the answer alone.
    Always return this: "Please provide a valid query." for empty query.
    """
    question_prompt = PromptTemplate(input_variables=["user_query", "doc_list"], template=template)
    initiator_router = question_prompt | llm | StrOutputParser()
    output = initiator_router.invoke({"user_query":user_query, "doc_list":doc_list})
    return output

In [11]:
answer_1 = medicalbot_ai(user_query="What is Candidiasis?", doc_list=search)

In [12]:
def medical_chatbot(user_query: str)-> str:
    logger.info("Searching for similar Docs in the Vec DB")
    doc_list = search_db(user_query=user_query)
    logger.info("Calling the medicalbot ai to get answer")
    answer = medicalbot_ai(user_query=user_query, doc_list=doc_list)
    logger.info("Returning the Final Answer")
    return answer

In [13]:
question_1 = "What is Candidiasis?"
question_2 = "what is Carpal tunnel syndrome?"
question_3 = "what is the treatment of Carpal tunnel syndrome?"
question_4 = "What is the treatment of Candidiasis?"

In [14]:
answer_1 = medical_chatbot(user_query=question_1)
print(answer_1)

2025-05-19 23:49:51.039 | INFO     | __main__:medical_chatbot:2 - Searching for similar Docs in the Vec DB
2025-05-19 23:49:51.338 | INFO     | __main__:medical_chatbot:4 - Calling the medicalbot ai to get answer
2025-05-19 23:49:52.141 | INFO     | __main__:medical_chatbot:6 - Returning the Final Answer


Unfortunately, I couldn't find any direct information about Candidiasis in the provided documents. However, I can tell you that the documents do mention fungi and antifungal drugs, which might be related to Candidiasis. Candidiasis is often caused by a type of fungus called Candida. If you're looking for information on Candidiasis, I'd be happy to try and help you find some general information on the topic, but it's not specifically mentioned in the documents provided. If you have any other questions related to the context of the documents, such as antifungal drugs or fungi, I'd be happy to try and assist you.


In [15]:
answer_2 = medical_chatbot(user_query=question_2)
print(answer_2)

2025-05-19 23:49:54.619 | INFO     | __main__:medical_chatbot:2 - Searching for similar Docs in the Vec DB
2025-05-19 23:49:54.985 | INFO     | __main__:medical_chatbot:4 - Calling the medicalbot ai to get answer
2025-05-19 23:49:55.282 | INFO     | __main__:medical_chatbot:6 - Returning the Final Answer


Carpal tunnel syndrome is a painful disorder that causes numbness or tingling in response to pressure on nerves around the wrist.


In [16]:
answer_3 = medical_chatbot(user_query=question_3)
print(answer_3)

2025-05-19 23:49:57.518 | INFO     | __main__:medical_chatbot:2 - Searching for similar Docs in the Vec DB
2025-05-19 23:49:57.866 | INFO     | __main__:medical_chatbot:4 - Calling the medicalbot ai to get answer
2025-05-19 23:49:58.282 | INFO     | __main__:medical_chatbot:6 - Returning the Final Answer


The provided documents do not contain information about the treatment of Carpal tunnel syndrome. I couldn't find any relevant information related to your query. Please consider asking a question related to arthritis, ataxia, or other topics mentioned in the documents.


In [17]:
answer_4 = medical_chatbot(user_query=question_4)
print(answer_4)

2025-05-19 23:50:00.917 | INFO     | __main__:medical_chatbot:2 - Searching for similar Docs in the Vec DB
2025-05-19 23:50:01.212 | INFO     | __main__:medical_chatbot:4 - Calling the medicalbot ai to get answer
2025-05-19 23:50:01.950 | INFO     | __main__:medical_chatbot:6 - Returning the Final Answer


The treatment of Candidiasis, also known as a fungal or yeast infection, can be approached in different ways. Alternative treatment focuses on creating an internal environment where the fungus cannot survive by eating a diet low in dairy products, sugars, and foods that contain yeast, and high in uncooked and unprocessed foods. Supplements of vitamins C, E, A-plus, and B complex, as well as Lactobacillus, may also be useful. For superficial skin infections, creams or ointments (topical antifungal drugs) are typically used. However, for systemic infections, yeast infections, or topical infections that do not clear up with creams or ointments, systemic antifungal drugs may be necessary.


In [18]:
test_5 = medical_chatbot("What is Deep Learning?")
print(test_5)

2025-05-19 23:50:04.847 | INFO     | __main__:medical_chatbot:2 - Searching for similar Docs in the Vec DB
2025-05-19 23:50:05.154 | INFO     | __main__:medical_chatbot:4 - Calling the medicalbot ai to get answer
2025-05-19 23:50:05.730 | INFO     | __main__:medical_chatbot:6 - Returning the Final Answer


Unfortunately, the provided documents do not contain information related to Deep Learning. The documents appear to be related to medical and neurological topics, but they do not mention Deep Learning. I apologize, but it seems that the documents do not have the necessary information to answer your question. Please consider asking a question related to the context of the provided documents, such as neurodegenerative diseases, brain structures, or medical imaging techniques.


In [19]:
test_5 = medical_chatbot("")
print(test_5)

2025-05-19 23:50:08.370 | INFO     | __main__:medical_chatbot:2 - Searching for similar Docs in the Vec DB
2025-05-19 23:50:08.721 | INFO     | __main__:medical_chatbot:4 - Calling the medicalbot ai to get answer
2025-05-19 23:50:08.969 | INFO     | __main__:medical_chatbot:6 - Returning the Final Answer


Please provide a valid query.
